In [1]:
from pathlib import Path

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import os
from tqdm import tqdm
import numpy as np

import torchvision.transforms as v1
from torchvision.io import read_image

plt.rcParams["savefig.bbox"] = 'tight'
torch.manual_seed(1)

In [2]:
!pip install mlconfig

In [3]:
import torch, torchvision
from torch.utils.data.dataloader import DataLoader
from sklearn.model_selection import train_test_split

import os, shutil
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import mlconfig
from importlib import import_module
from torch.utils.data import Dataset, DataLoader
import pandas as pd

def load_function(attr):
    module_, func = attr.rsplit('.', maxsplit=1)
    return getattr(import_module(module_), func)

config = mlconfig.load('config_resnet18.yaml')

In [4]:
class MyDataset(Dataset):
    def __init__(self, root_dir, csv_file):
        csv = pd.read_csv(csv_file)
        self.sig_filenames = csv["fragment_name"].values
        self.sig_labels = csv["label"].values
        self.root_dir = root_dir

    def __len__(self):
        return len(self.sig_labels)

    def __getitem__(self, idx):
        #with open(self.root_dir + '/' + self.sig_filenames[idx] + '.npy', 'r+') as fl:
         # data_file = np.frombuffer(fl.read(), dtype=np.float32)
        data_file = np.asarray(np.load(self.root_dir + '/' + self.sig_filenames[idx] + '.npy', 'r+'))
        if 'drone' in list(self.sig_filenames[idx].split('_')):
            label = torch.tensor(0)
        if 'noise' in list(self.sig_filenames[idx].split('_')):
            label = torch.tensor(1)
        if 'wifi' in list(self.sig_filenames[idx].split('_')):
            label = torch.tensor(2)
        return data_file, torch.tensor(label)

In [5]:
dataset = MyDataset(root_dir='S:/drone_dataset_v4.0_pictures_spectr', csv_file='S:/drone_dataset_v4.0_pictures_spectr/dataset.csv')
train_set, valid_set = torch.utils.data.random_split(dataset, [0.8, 0.2], generator=torch.Generator().manual_seed(42))
batch_size = config.batch_size
train_dataloader = torch.utils.data.DataLoader(
  train_set,
  batch_size=batch_size,
  shuffle=True,
  drop_last=True)
valid_dataloader = torch.utils.data.DataLoader(
  valid_set,
  batch_size=batch_size,
  drop_last=True,
  shuffle=True)

In [6]:
dataloaders = {}
dataloaders['train'] = train_dataloader
dataloaders['val'] = valid_dataloader
dataset_sizes = {}
dataset_sizes['train'] = len(train_set)
dataset_sizes['val'] = len(valid_set)
print(dict(dataloaders))

{'train': <torch.utils.data.dataloader.DataLoader object at 0x0000024C011E3E80>, 'val': <torch.utils.data.dataloader.DataLoader object at 0x0000024C011E3610>}


In [7]:
torch.cuda.empty_cache()
model = load_function(config.model.architecture)(pretrained=False)
model.fc = torch.nn.Linear(in_features=512, out_features=3, bias=True)
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)

import numpy as np
import copy, timeit

optimizer = load_function(config.optimizer.name)(model.parameters(), lr=config.optimizer.lr)
criterion = load_function(config.loss_function.name)()
scheduler = load_function(config.scheduler.name)(optimizer, step_size=config.scheduler.step_size, gamma=config.scheduler.gamma)

if device != 'cpu':
    model = model.to(device)

val_loss = []
val_acc = []
train_loss = []
train_acc = []
epochs = config.epoch

best_acc = 0
best_model = copy.deepcopy(model.state_dict())

start = timeit.default_timer()
for epoch in range(1, epochs+1):
    print(f"Epoch : {epoch}\n")
    dataloader = None

    for phase in ['train', 'val']:
        running_loss = 0.0
        running_corrects = 0

        for (img, label) in tqdm(dataloaders[phase]):
            #img = torch.unsqueeze(img,1)
            #label = torch.unsqueeze(label,1)
            img, label = img.to(device), label.to(device)
            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                output = model(img)
                _, pred = torch.max(output.data, 1)
                #print(output.shape)
                #print(output)
                #print(label.shape)
                #print(label)
                #loss = criterion(output, label)
                with torch.autocast('cuda'):
                    loss = criterion(output, label)
                if phase=='train' :
                    loss.backward()
                    optimizer.step()
            
            running_loss += loss.item() * img.size(0)
            running_corrects += torch.sum(pred == label.data)
            
        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
                
        if phase=='train' :
            train_loss.append(epoch_loss)
            train_acc.append(epoch_acc)
        else :
            val_loss.append(epoch_loss)
            val_acc.append(epoch_acc)
            if val_acc[-1] > best_acc :
                best_acc = val_acc[-1]
                best_model = copy.deepcopy(model.state_dict())
    print()

end = timeit.default_timer()
print(f"Total time elapsed = {end - start} seconds")
name = 'model2.pth'
model.load_state_dict(best_model)
model = model.to('cpu')
torch.save(model.state_dict(), name)

C:\Users\Admin\.conda\envs\python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Admin\.conda\envs\python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


cuda:0
Epoch : 1



  0%|                                                                                          | 0/572 [00:00<?, ?it/s]C:\Users\Admin\AppData\Local\Temp\ipykernel_16376\1625743276.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return data_file, torch.tensor(label)
100%|██████████████████████████████████████████████████████████████████████████████| 572/572 [2:10:24<00:00, 13.68s/it]


train Loss: 0.1438 Acc: 0.9511


100%|████████████████████████████████████████████████████████████████████████████████| 143/143 [08:29<00:00,  3.56s/it]


...

val Loss: 0.0098 Acc: 0.9974

Epoch : 5



100%|██████████████████████████████████████████████████████████████████████████████| 572/572 [2:11:33<00:00, 13.80s/it]


train Loss: 0.0095 Acc: 0.9971


100%|████████████████████████████████████████████████████████████████████████████████| 143/143 [08:14<00:00,  3.46s/it]

val Loss: 0.0071 Acc: 0.9983

Total time elapsed = 41664.194682400004 seconds


In [2]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(range(1,epochs+1), train_loss, color='blue')
plt.plot(range(1,epochs+1), val_loss, color='red')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curve')
plt.legend(['Train Loss', 'Validation Loss'])
plt.show()
plt.figure()
plt.plot(range(1,epochs+1), train_acc, color='blue')
plt.plot(range(1,epochs+1), val_acc, color='red')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy Curve')
plt.legend(['Train Accuracy', 'Validation Accuracy'])
plt.show()

NameError: name 'epochs' is not defined

<Figure size 640x480 with 0 Axes>